In [ ]:
from time_templates.fittemplate.fit_events import fit_events, fit_event
import matplotlib.pyplot as plt
from time_templates.datareader.get_data import get_event_from_df
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
import numpy as np
# plt.style.use('thesis')

In [ ]:
# df = pd.read_pickle('../data/df_observer_Golden_icrc19_test.pl')
df = pd.read_pickle('/home/mart/auger/data/time_templates/df_tree_observer_icrc19_Golden_SdlgE19_theta53_selected.pl')
df = df.query('Sdr_min > 500 & SdCosTheta > 0.6 & SdCosTheta < 0.9 & SdlgE > 19.1')

event = get_event_from_df(df, eventid=5184645, is_data=True, MC=False)#2984180, 25006220
m, TF = fit_event(event)

In [ ]:
TF.setup_scale_mask(tq_cut=0.95)
azim = event['SdAzimuth'][0]
f, axes = plt.subplots(3, 2, figsize=(6, 6))
ax1 = axes[0, 0]
ax2 = axes[0, 1]
TF.setup_scale_mask(tq_cut=0.95)

Deltat0s = TF.Deltat0s
t0_sigs = TF.t0_errs
t0s = [TT.t0 for TT in TF.TTTs]


xcore, ycore = event['SdCore.fX']/1000, event['SdCore.fY']/1000

x = [station.station_pos[0]/1000-xcore for station in event.stations]
y = [station.station_pos[1]/1000-ycore for station in event.stations]
s = [station.WCDTotalSignal for station in event.stations]
t = [station.TimeNS-event.SdCoreTimeNS for station in event.stations]

im = ax1.scatter(x, y, s=s, c=t, cmap='bwr')
arrowsize = 50
ax1.annotate(text='', xy=(0, 0), xytext=(arrowsize * np.cos(azim), arrowsize*np.sin(azim)), textcoords='offset points', arrowprops=dict(arrowstyle= '->'))

for station in event.stations:
    ax1.annotate(text=f'{station.stationid}', xy=(station.station_pos[0]/1000-xcore, station.station_pos[1]/1000-ycore), xytext=(-15, -10), textcoords='offset points')

ax1.plot(x[np.argmax(x)], y[np.argmax(y)], color='grey', ms=3, mfc='none', marker='o')
ax1.set_xlim([-2, 2])
ax1.set_ylim([-2.6, 2.4])
ax1.set_xlabel('$x - x_{\\rm c} $ [km]')
ax1.set_ylabel('$y - y_{\\rm c}$ [km]')

TF.fit_total_signals(ax=ax2)
ax2.set_ylabel('Signal [VEM]')

from time_templates.templates.universality.names import DICT_COMP_COLORS, DICT_COMP_LABELS
from matplotlib.lines import Line2D

handles = []
labels = []
from matplotlib.container import ErrorbarContainer
from matplotlib.collections import LineCollection
line = Line2D([],[], ls="none", color='k', marker='o')
barline = LineCollection(np.empty((2,0,2)), color='k')
err = ErrorbarContainer((line, None, [barline]), has_yerr=True)

handles.append(err)
labels.append('total signal')

handles.append(Line2D([0], [0], color='k', ls='--'))
labels.append('total model')

first_leg = ax2.legend(handles, labels, loc='upper left', fontsize=8)
handles = []
labels = []

for key in DICT_COMP_LABELS:
    handles.append(Line2D([0], [0], color=DICT_COMP_COLORS[key], ls='--'))
    labels.append(f"${DICT_COMP_LABELS[key]}$")
    
ax2.legend(handles, labels, ncol=1, fontsize=8, loc='upper right')
ax2.add_artist(first_leg)


import matplotlib
ax2.set_yticks([1, 10, 100, 1000, 10000])
locmin = matplotlib.ticker.LogLocator(base=10.0,subs=(0.2,0.4,0.6,0.8),numticks=12)
ax2.yaxis.set_minor_locator(locmin)
# ax2.yaxis.set_minor_formatter(matplotlib.ticker.NullFormatter())
ax2.set_ylim([0.5, 2e4])

#traces plot

indexes = [0, 2, 4, 5]
iax = 2
for i, station in enumerate(event.stations):
    if i not in indexes:
        continue
    ax = axes.flatten()[iax]
    iax += 1
    station.plot_trace(ax=ax, detector='wcd', plotTT=True, plotMC=False, infostr=False)
    ymax = ax.get_ylim()[1]
    yarrow = ymax/25
    ax.annotate(text='$\sigma_{t_0}$', xy=(t0s[i]-Deltat0s[i], -yarrow*2.8), ha='center', va='center', fontsize=8)
    ax.set_ylim((-yarrow*4.4, ymax))
    ax.errorbar(t0s[i]-Deltat0s[i], -1*yarrow, xerr=t0_sigs[i], xuplims=True, xlolims=False, color='k', marker='', lw=1)
    ax.errorbar(t0s[i]-Deltat0s[i], -1*yarrow, xerr=t0_sigs[i], xuplims=False, xlolims=True, color='k', marker='', lw=1)
    t = TF.ts[i]
    mask = TF.masks[i]
    tmax = np.max(t[mask])
    ax.axvspan(tmax, 5000, hatch='\\\\\\', fc='none', color='grey', lw=0)
    ax.set_xlim([-50, tmax*1.2])
    ax.annotate(text=f'ID = {station.stationid} \n $r = {station.r:.0f}$ m', xy=(0.5, 0.95), va='top', xycoords='axes fraction')
#     ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))
#     ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

axes[1, 1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
# axes[2, 0].set_yticks([0, 0.2, 0.4, 0.6, 0.8])
axes[2, 1].set_yticks([0, 0.1, 0.2, 0.3]) 

f.align_ylabels(axes[:, 1])
f.align_ylabels(axes[:, 0])
f.subplots_adjust(wspace=0.3, hspace=0.35)
f.savefig('hybrid_example_theta35_lgE19_2_Xmax761.pdf', bbox_inches='tight')

In [ ]:
df = pd.read_pickle('/home/mart/auger/data/time_templates/df_tree_MCTask_Offline_v3r99p2a_EPOSLHC_iron_19.0_19.5.pl')
df = df.query('Sdr_min > 500 & SdCosTheta > 0.6 & SdCosTheta < 0.9 & SdlgE > 19.1')

In [ ]:
event = get_event_from_df(df, eventid=None, is_data=False)# 1726240103, 1655260103, 1624507192
m, TF = fit_event(event)

In [ ]:
f, axes = plt.subplots(3, 2, figsize=(6, 6))
# axes = axes.flatten()

for station, ax in zip(event.stations, axes.flatten()):
    station.plot_trace(ax=ax, detector='wcd', plotTT=True, plotMC=True)
    ax.set_xlim([0, 3000])
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.yaxis.set_major_locator(plt.MaxNLocator(4))
    ax.xaxis.set_major_locator(plt.MaxNLocator(5))

# axes[1, 1].set_yticks([0, 0.5, 1, 1.5, 2.0])
# axes[2, 1].set_yticks([0, 0.2, 0.4, 0.6, 0.8])
f.align_ylabels(axes[:, 1])
f.align_ylabels(axes[:, 0])
f.subplots_adjust(wspace=0.3, hspace=0.35)
f.savefig('MC_example_EPOS_iron_theta32_lgE19_1.pdf', bbox_inches='tight')

In [ ]:
axes.shape